# Noisy 1B5 (Finetune Training)
We will be finetuning a 1B5 model, and intentionally overtrain it.
This is to measure how the introduction of a noisy state will impact train.

The goal of this experiment, is to measure two seperate objective
- test for overfitting conditions across multiple epoch
- test if overfitting is mitigated with the introduction of a noisy state

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps

In [1]:
# First lets get the RWKV base model which we will be finetuning
!mkdir -p ../../../model/
!mkdir -p ../../../datapath/
!mkdir -p ../../../checkpoint/
!cd ../../../model/ && wget https://huggingface.co/BlinkDL/rwkv-4-pileplus/resolve/main/RWKV-4-PilePlus-1B5-20230520-2942-486Gtokens-ctx4096.pth

--2023-07-18 13:31:30--  https://huggingface.co/BlinkDL/rwkv-4-pileplus/resolve/main/RWKV-4-PilePlus-1B5-20230520-2942-486Gtokens-ctx4096.pth
Resolving huggingface.co (huggingface.co)... 143.204.55.75, 143.204.55.124, 143.204.55.121, ...
Connecting to huggingface.co (huggingface.co)|143.204.55.75|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/a0/d0/a0d028cbd69f2f788f85a9e3b66d39102ab8e09500502ba8b9780cd52652da12/d3ca6ee3a10b4586b2dff826e3c99dbfa28ecaa71596e82e59a1948c38029e40?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27RWKV-4-PilePlus-1B5-20230520-2942-486Gtokens-ctx4096.pth%3B+filename%3D%22RWKV-4-PilePlus-1B5-20230520-2942-486Gtokens-ctx4096.pth%22%3B&Expires=1689946290&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY4OTk0NjI5MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9hMC9kMC9hMGQwMjhjYmQ2OWYyZjc4OGY4NWE5ZTNiNjZkMzkxMDJhYjhlM

In [2]:
DEEPSPEED_STRAT="deepspeed_stage_1"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="MultiEpoch-1B5"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v4neo/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

DEEPSPEED_STRAT: deepspeed_stage_1
ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /root/picocreator-noise-experiment/notebook/experiment/noisy-training
TRAINER_DIR: /root/picocreator-noise-experiment/RWKV-v4neo
PROJECT_DIR: /root/picocreator-noise-experiment


In [3]:
# Lets preload the requried dataset (enwiki_100k)
!cd "{TRAINER_DIR}" && \
    python3 preload_dataset.py "{NOTEBOOK_DIR}/MultiEpoch-1B5-instruct-baseline.yaml"

Extracting data files: 100%|████████████████████| 1/1 [00:00<00:00, 1589.35it/s]
Setting num_proc from 64 back to 1 for the train split to disable multiprocessing as it only contains one shard.
Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/c-s-ale___parquet/c-s-ale--dolly-15k-instruction-alpaca-format-9dfbb23260d63d9d/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 684.23it/s]


## Stage 1 : Baseline model training

In [4]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python new_train.py fit \
        -c "{NOTEBOOK_DIR}/MultiEpoch-1B5-instruct-baseline.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Instruct Tune Baseline (ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" 

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/fabric/utilities/seed.py:39: UserWarning: No seed found, seed set to 3988496179
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 3988496179
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230718_133306-2xjrwkm9
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run MultiEpoch-1B5 - Instruct Tune Baseline (ctx=4096, deepspeed_stage_1)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-Noise-Training
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-Noise-Training/runs/2xjrwkm9
Using /root/.cache/torch_extensions/py311_cu118 as PyTor

In [5]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/MultiEpoch-1B5-Baseline/last.ckpt" "../model/MultiEpoch-1B5-Baseline.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/MultiEpoch-1B5-Baseline.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/MultiEpoch-1B5-Baseline/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 438 params 1515106304 elements
Saving fp32 state dict to ../model/MultiEpoch-1B5-Baseline.pth
-rw-r--r-- 1 root root 5.7G Jul 18 21:01 ../model/MultiEpoch-1B5-Baseline.pth


## Stage 2 : Noisy model training

In [6]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python new_train.py fit \
        -c "{NOTEBOOK_DIR}/MultiEpoch-1B5-instruct-noisy.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Instruct Tune Noisy (ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" 

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/fabric/utilities/seed.py:39: UserWarning: No seed found, seed set to 2781093071
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 2781093071
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230718_210139-g2nx0ro6
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run MultiEpoch-1B5 - Instruct Tune Noisy (ctx=4096, deepspeed_stage_1)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-Noise-Training
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-Noise-Training/runs/g2nx0ro6
Using /root/.cache/torch_extensions/py311_cu118 as PyTorch 

In [7]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/MultiEpoch-1B5-Noisy/last.ckpt" "../model/MultiEpoch-1B5-Noisy.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/MultiEpoch-1B5-Noisy.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/MultiEpoch-1B5-Noisy/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 438 params 1515106304 elements
Saving fp32 state dict to ../model/MultiEpoch-1B5-Noisy.pth
-rw-r--r-- 1 root root 5.7G Jul 19 05:32 ../model/MultiEpoch-1B5-Noisy.pth
